In [5]:
import pandas as pd
import openai
import requests
import json
import re
import time
import faiss
import ast
import numpy as np
from tqdm import tqdm
from openai import OpenAI

tqdm.pandas(desc="Example Desc")

In [2]:
with open('data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [6]:
def convert_to_list(text):
    return ast.literal_eval(text)

LAW_FILE_PATH = '/home/monsh/works/hackathon2/new_laws.csv'

global law_data
law_data = pd.read_csv(LAW_FILE_PATH)
law_data['ada_embedding'] = law_data['ada_embedding'].apply(convert_to_list)
law_data['keywords_embbed'] = law_data['keywords_embbed'].apply(convert_to_list)

GPT_API_KEY = "<token>"

In [7]:
def get_embedding(text, model="text-embedding-3-large"): # change to large model!
   client = openai.OpenAI(api_key=GPT_API_KEY)
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def find_related_content(input_text, data_df, k=4, dimension = 3072, distance_threshold=1.23, column='ada_embeddings'): #3072 for large embedding model
    data_np = np.array(list(data_df[column].values))
    index = faiss.IndexFlatL2(dimension)
    index.add(data_np)
    input_vector = get_embedding(input_text)
    distances, indices = index.search(np.expand_dims(input_vector, axis=0), k)

    #print dis
    # for dist, ind in zip(distances[0], indices[0]):
    #     print(ind, "dist:", dist)

    # Filtering results based on distance threshold
    filtered_distances = []
    filtered_indices = []
    for dist, ind in zip(distances[0], indices[0]):
        if dist <= distance_threshold:
            filtered_distances.append(dist)
            filtered_indices.append(ind)


    return np.array(filtered_distances), np.array(filtered_indices)

def search_law(text, law_data=law_data, column='ada_embedding'):
    # global law_data
    dis, ind = find_related_content(text, law_data, column=column)

    formatted_string = ""
    related_articles = "\n\nกฏหมายที่เกี่ยวข้อง:\n"
    related_articles_dict = {}
    for index in ind:
        row = law_data.iloc[index]
        formatted_string += f"{row['หัวข้อ']} : {row['มาตรา']}, : {row['เนื้อหา']}\n"

        if row['หัวข้อ'] not in related_articles_dict:
            related_articles_dict[row['หัวข้อ']] = [row['มาตรา']]
        else:
            related_articles_dict[row['หัวข้อ']].append(row['มาตรา'])

    if len(related_articles_dict) == 0:
        related_articles = ""

    for key, value in related_articles_dict.items():
        related_articles += f"{key} : {', '.join(value)}\n"
    
    return formatted_string, related_articles

In [68]:
def get_answer(system_prompt, instructions_dict, model='llama3-70b-typhoon', max_tokens=2048):
    response = requests.post("https://kjddazcq2e2wzvzv.snova.ai/api/v1/chat/completion", 
        headers={
            "Authorization": "Basic <token>",
            "Content-Type": "application/json"
        },
        json={
            "inputs": [
                {
                    "role": "system",
                    "content": system_prompt
                },
                *instructions_dict
            ],
            # "max_tokens": max_tokens,
            "stop": ["<|eot_id|>", "<|end_of_text|>"],
            "model": model,
            "stream": False
        }
    )
    # print(response.text)
  
    stop = False
    data = None
    for i in response.iter_lines():
        if stop:
            # print(i)
            data = i.decode('unicode_escape').replace("data: ", "")
            break

        if i.decode('utf-8') == 'event: end_event':
            stop = True
    if data is None:
        return None
    
    start_string = "\"completion\""
    end_string = ", \"stream_token\":"

    text = str(data)
    start_index = text.find(start_string)
    end_index = text.find(end_string, start_index + len(start_string) + 3)
    text = text[start_index + len(start_string) + 3: end_index-1].strip()#.replace("\\\\", "\\")#.replace("\\n", "\n")

    return text

def find_law_from_text(text):
    laws = re.findall('มาตรา [0-9]+', text)
    return laws

def get_descriptive_laws(json_laws):
    if len(json_laws) == 1 and json_laws[0]['มาตรา'] == -1:
        return json_laws[0]['เหตุผล']
    
    descriptive_laws = []
    for law in json_laws:
        descriptive_laws.append(f"มาตรา {law['มาตรา']} : {law['เหตุผล']}")

    return "\n".join(descriptive_laws)

def typhoon(system, prompt, select_model='typhoon-v1.5x-70b-instruct', temperature=0.35):
    client = OpenAI(
        api_key='<token>',
        base_url="https://api.opentyphoon.ai/v1",
    )
    response = client.chat.completions.create(
    model=select_model,
    messages=[
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ],
    temperature=temperature
    )
    return response.choices[0].message.content


In [28]:
def module_1(text, related_articles, select_model="typhoon-v1.5-8b-instruct", temperature=0.35):
    """Law pattern Extraction"""

    if select_model == "llama3-8b-typhoon" or select_model == "llama3-70b-typhoon":
        prepare_json = get_answer(
            """
            You're Thai lawyer.
            User will give you an scenario you should extract the scenario into steps.
            """,
            [{
                "role": "user", 
                "content": """
                    The scenario will be in plain-text format.
                    คุณจะต้องอธิบายว่าแต่ละมาตราเกี่ยวข้องหรือไม่เกี่ยวข้องกับเหตุการณ์ที่กล่าวถึงอย่างไร
                    
                    มาตราที่พบ:
                    ```
                    {มาตราที่พบ}
                    ```

                    เหตุการณ์ หรือ คำถาม:
                    ```
                    {เหตุการณ์}
                    ```

                    Instructions:
                        - อธิบายความเชื่อมโยง แต่ละมาตรา กับเหตุการณ์ที่กล่าวถึง
                        - มาตราที่ไม่เกี่ยวข้องควรจะน้อย
                        - Don't use other 'มาตรา' rather the given.
                        - Please carefully about number of 'มาตรา' in the answer.
                    
                    Format:
                        - มาตรา .. : ข้อความเกี่ยวข้อง
                """.replace("{มาตราที่พบ}", related_articles).replace("{เหตุการณ์}", text),
            },],
            model=select_model,
        )
    else:
        prepare_json = typhoon(
            """
            You're Thai lawyer.
            User will give you an scenario you should extract the scenario into steps.  
            """, 
            """
                The scenario will be in plain-text format.
                คุณจะต้องอธิบายว่าแต่ละมาตราเกี่ยวข้องหรือไม่เกี่ยวข้องกับเหตุการณ์ที่กล่าวถึงอย่างไร
                
                มาตราที่พบ:
                ```
                {มาตราที่พบ}
                ```
                เหตุการณ์ หรือ คำถาม:
                ```
                {เหตุการณ์}
                ```

                Instructions:
                    - อธิบายความเชื่อมโยง แต่ละมาตรา กับเหตุการณ์ที่กล่าวถึง
                    - มาตราที่ไม่เกี่ยวข้องควรจะน้อย
                    - Don't use other 'มาตรา' rather the given.
                    - Please carefully about number of 'มาตรา' in the answer.
                
                Format:
                    - มาตรา .. : ข้อความเกี่ยวข้อง
            """.replace("{มาตราที่พบ}", related_articles).replace("{เหตุการณ์}", text),
            select_model,
            temperature
        )
    return prepare_json
    
def module_2(text, format_laws, select_model="typhoon-v1.5-8b-instruct", temperature=0.35):
    """Law pattern Extraction"""

    if select_model == "llama3-8b-typhoon" or select_model == "llama3-70b-typhoon":
        answer = get_answer(
            """
                You're Law expert.
                You can analysis the given scenario into informative Thai law based academic article.
                - คุณจะไม่ถามข้อมูลเพิ่มเติมจากผู้ใช้ หรือตอบแบบปลายเปิด
            """,
            [{
                "role": "user", 
                "content": """
                    Analysis the given scenario
                    
                    The scenario:
                    ```
                    {scenario}
                    ```
                    Consider these as a law information:
                    {infomation}

                    Instructions:
                        - Analysis the given scenario and information in TERM OF LAW. (Who, doing what, why, law)
                        - Thai language.
                        - YOU SHOULD NOT ASK THE USER FOR MORE INFORMATION. (no question)
                        - Don't use other 'มาตรา' rather the given.

                    Example แนวทางการเขียนเหตุผล:
                    ```
                    นาย A ถูกกล่าวหาว่ากระทำชำเรา B ซึ่งมีอายุระหว่าง .... การกระทำดังกล่าวถือว่าเป็นการกระทำที่ผิดกฎหมาย ตามประมวลกฎหมาย... มาตรา ...
                    ```
                    ```
                    แม้ว่าเด็กชาย C และไม่ได้ส่งตัวพี่สาว ไปตรวจหาร่องรอยการกระทำชำเราทันที ตามประมวลวิธีพิจารณาความอาญา มาตรา ...
                    ```
                """.replace("{scenario}", text).replace("{infomation}", format_laws),},],
                model=select_model,
        )
    else:
        answer = typhoon(
            """
                You're Law expert.
                You can analysis the given scenario into informative Thai law based academic article.
                - คุณจะไม่ถามข้อมูลเพิ่มเติมจากผู้ใช้ หรือตอบแบบปลายเปิด
            """,
            """
                Analysis the given scenario
                
                The scenario:
                ```
                {scenario}
                ```
                Consider these as a law information:
                {infomation}

                Instructions:
                    - Analysis the given scenario and information in TERM OF LAW. (Who, doing what, why, law)
                    - Thai language.
                    - YOU SHOULD NOT ASK THE USER FOR MORE INFORMATION. (no question)
                    - Don't use other 'มาตรา' rather the given.

                Example แนวทางการเขียนเหตุผล:
                ```
                นาย A ถูกกล่าวหาว่ากระทำชำเรา B ซึ่งมีอายุระหว่าง .... การกระทำดังกล่าวถือว่าเป็นการกระทำที่ผิดกฎหมาย ตามประมวลกฎหมาย... มาตรา ...
                ```
                ```
                แม้ว่าเด็กชาย C และไม่ได้ส่งตัวพี่สาว ไปตรวจหาร่องรอยการกระทำชำเราทันที ตามประมวลวิธีพิจารณาความอาญา มาตรา ...
                ```
            """.replace("{scenario}", text).replace("{infomation}", format_laws),
            select_model,
            temperature
        )
    return answer

def extract_keyword_from_query(system_prompt, query, select_model="typhoon-v1.5-8b-instruct", temperature=0.35):
    if select_model == "llama3-8b-typhoon" or select_model == "llama3-70b-typhoon":
        keywords = get_answer(
            system_prompt, 
            [{
                "role": "user", 
                "content": """
                    Extract {number} keywords from this
                    ```
                    {query}
                    ```

                    Instruction:
                        - The {number} keywords should be the content of the query.
                        - Don't repeat the same keywords.
                        - The keywords should be law content.
                        - Answer only the keywords.
                
                    Example:
                        - ทำร้ายร่างกาย, คดีอาญา, การสอบสวน, โดนโกง, ลูกจ้าง
                """.format(query=query, number=8)
            }],
            model='llama3-8b-typhoon', 
            max_tokens=2048
        )
    else:
        keywords = typhoon(
            system_prompt, 
            """
                Extract {number} keywords from this
                ```
                {query}
                ```

                Instruction:
                    - The {number} keywords should be the content of the query.
                    - Don't repeat the same keywords.
                    - The keywords should be law content.
                    - Answer only the keywords.

                Example:
                    - ทำร้ายร่างกาย, คดีอาญา, การสอบสวน, โดนโกง, ลูกจ้าง
            """.format(query=query, number=8),
            select_model,
            temperature
        )

    return keywords

In [21]:
systems_prompt_dict = {
    "summary": (
        "You're Thai summarizer\n"
        "- You need to summarize what user what\n\n" 
    ),
    "keyword": "You're Thai keyword extractor\n",
    "main": """
        คุณชื่อ บัดดี้ (LawBuddy)
        - คุณเป็นทะนาย
        - คุณมักจะตอบคำถามด้วยภาษามนุษย์เข้าใจง่าย
        - ไม่ใช้ . , ? ในการตอบคำถาม
    """,
    "reflection": "You're Thai lawyer",
    "classification": "You're Law type classifier",
}

In [22]:
with open('../question.txt', 'r', encoding="utf-8") as f:
    queries = f.read()
queries = queries.split("-----")
queries[0]

'นายเอกสมรสกับนางโทมีบุตรด้วยกัน 1 คน คือ เด็กหญิงตรี ต่อมานายเอกจดทะเบียนหย่ากับนางโท เด็กหญิงตรีจึง อาศัยอยู่กับนายเอก เมื่อเด็กหญิงตรีอายุ 14 ปี นายเอกพาเด็กหญิงตรีไปเที่ยวที่บ้านนายจัตวาและนอนค้างคืนที่บ้านนายจัตวา หลังจากนายเอกนอนหลับ นายจัตวากระทําชําเราเด็กหญิงตรี วันรุ่งขึ้นเด็กหญิงตรีจึงร้องทุกข์ต่อพนักงานสอบสวนโดยลําพัง\nเพื่อให้ดําเนินคดีอาญาแก่นายจัตวา ในชั้นสอบสวนการสอบปากคําเด็กหญิงตรีผู้เสียหาย พนักงานสอบสวนจัดให้มีนักจิตวิทยา หรือนักสังคมสงเคราะห์ นายเอกซึ่งเป็นบุคคลที่เด็กหญิงตรีร้องขอและพนักงานอัยการร่วมอยู่ด้วย โดยแยกกระทําเป็นส่วนสัดใน สถานที่ที่เหมาะสมแล้ว แต่พนักงานสอบสวนไม่ได้ตั้งคําถามเด็กหญิงตรีผู้เสียหายผ่านนักจิตวิทยาหรือนักสังคมสงเคราะห์ อีกทั้ง วันรับแจ้งเหตุพนักงานสอบสวนมิได้ส่งตัวเด็กหญิงตรีผู้เสียหายไปตรวจหาร่องรอยการกระทําชําเราเป็นพยานหลักฐานทันที โดยให้แพทย์ตรวจร่างกายเด็กหญิงตรีผู้เสียหายหลังเกิดเหตุ 3 วัน ต่อมาพนักงานสอบสวนสรุปสํานวนส่งไปยังพนักงานอัยการ และพนักงานอัยการฟ้องนายจัตวาเป็นจําเลยต่อศาลขอให้ลงโทษตามประมวลกฎหมายอาญา มาตรา 277\nให้วินิจฉัย

In [69]:
template = """
    Possible related laws: 
        {related}

    Question: 
        {question}
  
    Instruction:
        - Please answer the question in Thai language.
        - ตอบอย่างเป็นธรรมชาติ (natural language)
        - Answer in detail.
"""

In [70]:
answer = []

for query in tqdm(queries):
    keywords = extract_keyword_from_query(systems_prompt_dict['keyword'], query, select_model="typhoon-v1.5-instruct")

    possible_laws, references_laws = search_law(keywords, column='keywords_embbed')
    possible_laws2, references_laws2 = search_law(query)
    possible_laws += "\n" + possible_laws2

    json_laws = module_1(query, possible_laws, select_model="typhoon-v1.5-instruct") # "llama3-70b-typhoon")
    article = module_2(query, json_laws, select_model="typhoon-v1.5-instruct") # select_model="llama3-70b-typhoon")
    # print(article)

    # article = typhoon(
    #     "Your Thai lawyer",
    #     template.format(related=references_laws2, question=query),
    #     select_model="llama3-70b-typhoon"
    # )
    article = get_answer(
        "You're Thai lawyer",
        [{"role": "user", "content": template.format(related=references_laws2, question=query)}],
        model='llama3-70b-typhoon'
    )
    print(article)
    answer.append(article)

    # OLD Code

    # if len(query) < 150:
    #     prompt += "\n\t- Short simple answer is enough except for the law articles."
    # else:
    #     prompt += """
    #         - Please answer in detail.
    #         - Analysis the situation and provide the answer.

    #         Example TERM OF LAW:
    #         ```
    #         นาย A ถูกกล่าวหาว่ากระทำชำเรา B ซึ่งมีอายุระหว่าง .... การกระทำดังกล่าวถือว่าเป็นการกระทำที่ผิดกฎหมาย ตามประมวลกฎหมาย... มาตรา ...
    #         ```
    #         ```
    #         แม้ว่า... และไม่ได้ส่งตัว... ไปตรวจหาร่องรอยการกระทำชำเราทันที ตามประมวลวิธีพิจารณาความอาญา มาตรา ...
    #         ```
    #     """

    # response = get_answer(
    #     systems_prompt_dict['main'],
    #     [{"role": "user", "content": prompt}])
    #
    # answer.append(response)
    # print(response)
    # break

 10%|█         | 1/10 [00:05<00:52,  5.82s/it]

การร้องทุกข์ของเด็กหญิงตรีผู้เสียหายชอบด้วยกฎหมายหรือไม่

ตามประมวลกฎหมายวิธีพิจารณาความอาญา มาตรา 14 ระบุว่า "ผู้เสียหายมีสิทธิ์ร้องทุกข์ต่อพนักงานสอบสวน" ดังนั้น การร้องทุกข์ของเด็กหญิงตรีผู้เสียหายจึงชอบด้วยกฎหมาย

การสอบสวนชอบด้วยกฎหมายหรือไม่

การสอบสวนไม่ชอบด้วยกฎหมาย เนื่องจากพนักงานสอบสวนไม่ได้ตั้งคําถามเด็กหญิงตรีผู้เสียหายผ่านนักจิตวิทยาหรือนักสังคมสงเคราะห์ ซึ่งขัดต่อประมวลกฎหมายวิธีพิจารณาความอาญา มาตรา 52 ที่ระบุว่า "ในการสอบสวนผู้เสียหายที่เป็นเด็กหรือผู้ที่มีความบกพร่องทางจิตใจ พนักงานสอบสวนต้องจัดให้มีนักจิตวิทยาหรือนักสังคมสงเคราะห์เข้าร่วมสอบสวน" 

นอกจากนี้ พนักงานสอบสวนยังไม่ได้ส่งตัวเด็กหญิงตรีผู้เสียหายไปตรวจหาร่องรอยการกระทําชําเราเป็นพยานหลักฐานทันที ซึ่งขัดต่อประมวลกฎหมายวิธีพิจารณาความอาญา มาตรา 182 ที่ระบุว่า "เมื่อพนักงานสอบสวนได้รับแจ้งเหตุแล้ว ต้องส่งตัวผู้เสียหายไปตรวจหาร่องรอยการกระทําชําเราเป็นพยานหลักฐานทันที"

พนักงานอัยการมีอํานาจฟ้องหรือไม่

พนักงานอัยการมีอํานาจฟ้องนายจัตวาเป็นจําเลยต่อศาลขอให้ลงโทษตามประมวลกฎหมายอาญา มาตรา 277 เนื่องจากพนักงานอัยก

 20%|██        | 2/10 [00:10<00:43,  5.39s/it]

ข้อต่อสู้ของนายอ่อนฟังขึ้นไม่ได้

ตามที่ปรากฏในรายงานประจําวันของร้อยตํารวจเอกแข็ง นายอ่อนและนายเหนียวได้มาที่สถานีตํารวจเพื่อตกลงยอมความ โดยนายอ่อนมอบเงินให้ 100,000 บาท และส่วนที่เหลือจะจ่ายกันภายใน 1 เดือน เมื่อจ่ายเงินครบ 200,000 บาท นายเหนียวจะถอนคําร้องทุกข์ให้ ซึ่งแสดงว่ามีการยอมความเกิดขึ้นแล้ว

ตามประมวลกฎหมายวิธีพิจารณาความแพ่ง มาตรา๑๙๖ การยอมความเป็นการยุติคดีโดยไม่ต้องมีการพิจารณาของศาล และเมื่อมีการยอมความแล้ว คดีจะถือว่าสิ้นสุดลง

ในกรณีนี้ นายอ่อนได้ปฏิบัติตามข้อตกลงยอมความโดยมอบเงินให้ 100,000 บาท และเตรียมเงินที่เหลือเพื่อชําระตามที่ได้ตกลงกัน แต่นายเหนียวกลับปฏิเสธไม่รับเงินและขอให้ดําเนินคดีแก่นายอ่อนต่อไป ซึ่งไม่สอดคล้องกับข้อตกลงยอมความที่ได้ทำไว้

ดังนั้น ข้อต่อสู้ของนายอ่อนที่ว่า นายเหนียวจะปฏิเสธไม่รับเงินไม่ได้ และคดีมีการยอมความและสิทธินําคดีอาญามาฟ้องระงับไปแล้ว จึงฟังขึ้นไม่ได้


 30%|███       | 3/10 [00:20<00:52,  7.45s/it]

...

ตามประมวลกฎหมายวิธีพิจารณาความอาญา มาตรา๑๗๓/๒ กำหนดว่า "ในกรณีที่โจทก์ไม่นำพยานเอกสารหรือพยานวัตถุที่ระบุไว้ในบัญชีระบุพยานและอยู่ในความครอบครองของโจทก์มาส่งต่อศาลเพื่อให้จําเลยตรวจสอบภายในเวลาที่ศาลกำหนด ศาลอาจไม่รับฟังพยานเอกสารหรือพยานวัตถุนั้น"

ดังนั้น ในกรณีที่โจทก์ไม่นำพยานเอกสารและพยานวัตถุที่ระบุไว้ในบัญชีระบุพยานและอยู่ในความครอบครองของโจทก์มาส่งต่อศาลเพื่อให้จําเลยตรวจสอบภายในเวลาที่ศาลกำหนด ศาลมีสิทธิ์ที่จะไม่รับฟังพยานเอกสารและพยานวัตถุนั้น

แต่ศาลยังมีสิทธิ์ที่จะรับฟังพยานเอกสารและพยานวัตถุนั้นได้ หากศาลเห็นว่ามีเหตุผลที่สมควร เช่น โจทก์มีเหตุผลที่สมควรในการไม่สามารถนำพยานเอกสารและพยานวัตถุมาส่งต่อศาลได้ หรือพยานเอกสารและพยานวัตถุนั้นมีความสำคัญต่อการพิจารณาคดี

ดังนั้น ศาลจะพิจารณาในแต่ละกรณีโดยอาศัยหลักฐานและเหตุผลที่โจทก์นำเสนอ และจะวินิจฉัยว่าจะรับฟังพยานเอกสารและพยานวัตถุของโจทก์หรือไม่


 90%|█████████ | 9/10 [00:55<00:05,  5.95s/it]

คําร้องขอสืบพยานปากนายทอง เดียวดาย ไว้ก่อนเสียชีวิต

เรื่อง ขอสืบพยานปากนายทอง เดียวดาย ไว้ก่อนเสียชีวิต

โจทก์ นายอังคาร สีทอง ขอแสดงความเคารพต่อศาลและขอยื่นคําร้องขอสืบพยานปากนายทอง เดียวดาย ไว้ก่อนเสียชีวิต ดังนี้

1. ตามที่ปรากฏในคดีนี้ โจทก์ได้นําสัญญากู้เงินฉบับหนึ่งมาเป็นหลักฐานในการฟ้องเรียกเงินคืนจากจําเลย นายจันทร์ ส่องแสง โดยมีนายทอง เดียวดาย เป็นพยานในฐานะผู้ลงชื่อในฐานะพยานในสัญญากู้เงินดังกล่าว
2. แต่เนื่องจากนายทอง เดียวดาย ป่วยเป็นมะเร็งระยะสุดท้าย และแพทย์ลงความเห็นว่ามีชีวิตอยู่ต่อได้ไม่เกิน 1 เดือน เท่านั้น โจทก์จึงเกรงว่านายทอง เดียวดาย จะเสียชีวิตก่อนเป็นพยานในชั้น
3. ดังนั้น โจทก์จึงขอสืบพยานปากนายทอง เดียวดาย ไว้ก่อนเสียชีวิต เพื่อไม่ให้สูญเสียพยานที่สำคัญในคดีนี้ และเพื่อให้การพิจารณาคดีเป็นไปอย่างถูกต้องและเป็นธรรม

ด้วยเหตุผลข้างต้น โจทก์จึงขอสืบพยานปากนายทอง เดียวดาย ไว้ก่อนเสียชีวิต และขอให้ศาลพิจารณาอนุญาตตามที่โจทก์ขอร้อง

ขอแสดงความเคารพ

(ชื่อ) นายอังคาร สีทอง
โจทก์


100%|██████████| 10/10 [01:00<00:00,  6.04s/it]

ส่วนที่ 1

1. รัฐธรรมนูญ
2. การแก้ปัญหาโดยวิธีอื่น
3. การขโมยตัวตน
4. หนังสือบริคณห์สนธิ

ส่วนที่ 2

5. ศาลปกครองเป็นองค์กรหลักของประเทศที่มีอำนาจในการพิจารณาและตัดสินคดีปกครอง
6. การอุทธรณ์คือกระบวนการขอรื้อฟื้นคำพิพากษาของศาลชั้นต้น
7. ถ้าโจรไม่ยอมรับความผิด, ตำรวจจะต้องพิสูจน์ความผิดนั้นอย่างแน่นอน
8. คณะกรรมการบริษัทมีหน้าที่ต้องปฏิบัติหน้าที่ตามกฎหมายด้วยความซื่อสัตย์

ส่วนที่ 3

คำตอบที่ถูกต้องคือ B. Nation

เหตุผล: รัฐประกอบด้วยสี่องค์ประกอบ ได้แก่ รัฐบาล, อาณาเขต, ประชากร, และอธิปไตย ถ้าขาดองค์ประกอบใดองค์ประกอบหนึ่ง จะไม่สามารถเรียกว่ารัฐได้ แต่ชาติสามารถนิยามได้ว่าเป็นประชากรที่มีวัฒนธรรมและอุดมการณ์เหมือนกัน ชาติเกิดขึ้นจากการมีเชื้อชาติ, ศาสนา, ภาษา, อาณาเขต, ประวัติศาสตร์, วัฒนธรรม, หรือความปรารถนาทางการเมืองร่วมกัน องค์ประกอบเหล่านี้ไม่จำเป็นและเปลี่ยนแปลงอยู่เสมอ ดังนั้น ชาติไม่ใช่องค์ประกอบของรัฐ


## For validate

In [45]:
# from openai import OpenAI
# import anthropic


# def validate(prediction, actual, select_model, api, url):
#     """Law pattern Extraction"""
    
#     client = OpenAI(
#         api_key=api, base_url=url,
#     )

#     response = client.chat.completions.create(
#         model=select_model,
#         messages=[
#             {
#                 "role": "system",
#                 "content": """
#                     You're Thai lawyer.
#                     You analysis the given scenario.
#                 """
#             },
#             {
#                 "role": "user",
#                 "content": """
#                     You task is to scoring the answer based on the answer and the actual answer.
#                     The word 'ธงคําตอบ' is the answer but can be flexibled meaning.

#                     ธงคําตอบ:
#                     ```
#                     {prediction}
#                     ```

#                     เฉลยธงคําตอบ:
#                     ```
#                     {actual}
#                     ```
                    
#                     Scoring Criteria:
#                         1. ตอบไม่ถูกธงคําตอบ  และเหตุผลใช้ไม่ได้ (0 score)
#                         2. ตอบไม่ถูกธงคําตอบ  แต่เหตุผลพอฟังได้ (1-2 score)
#                         3. ตอบไม่ถูกธงคําตอบ  แต่เหตุผลดี (2-4 score)
#                         4. ตอบถูกธงคําตอบ  แต่เหตุผลใช้ไม่ได้ (0-1 score)
#                         5. ตอบถูกธงคําตอบ  และเหตุผลพอฟังได้บ้าง (2-5 score)
#                         6. ตอบถูกธงคําตอบ  และเหตุผลพอใช้ได้ (5-6 score)
#                         7. ตอบถูกธงคําตอบ  และเหตุผลดี (7-8 score)
#                         8. ตอบถูกธงคําตอบ  และเหตุผลดีมาก (9-10 score)
                        
#                     Instructions:
#                         - Scoring based on the given criteria.
#                         - Don't saying the answer is correct or not.
#                         - Just scoring the answer based on the given criteria.
#                         - If you provide multiple answers, please do mean of the scores.

#                     Output Format:
#                     ```
#                     <score> คะแนน
#                     ```
                
#                 """.replace("{prediction}", prediction).replace("{actual}", actual)
#             }
#         ],
#         temperature=0,
#         top_p=1
#     )
#     return response.choices[0].message.content


In [38]:
# from tqdm import tqdm
# import time
# import re

# with open("../test.txt", "r") as f:
#     test_data = f.read()

# test_data = test_data.split("-----")


# def calculate_score(text):
#     score = re.findall(r"\d+", text)
#     return int(score[0])

# with open("/home/monsh/works/answer_v2_typhoon.txt", "r") as f:
#     answer_typhoon_old = f.read()

# answer_typhoon_old = answer_typhoon_old.split("-----")

In [98]:
# score_list = []
# answer_list = []


# for groundtruth, query in tqdm(list(zip(test_data, answer))):
#     # score = validate(
#     #     query,
#     #     groundtruth,
#     #     select_model='gpt-3.5-turbo', 
#     #     api="<token>",
#     #     url=None
#     # ) 

#     score = validate(
#         query,
#         groundtruth,
#         select_model='gpt-4o', 
#         api="<token>",
#         url=None
#     )
    
#     print(score)
#     answer_list.append(answer)
#     score_list.append(score)

# print(sum([calculate_score(score) for score in score_list]))

In [91]:
print(answer[0])

การร้องทุกข์ของเด็กหญิงตรีผู้เสียหายชอบด้วยกฎหมายหรือไม่

ตามประมวลกฎหมายวิธีพิจารณาความอาญา มาตรา 14 ระบุว่า "ผู้เสียหายมีสิทธิ์ร้องทุกข์ต่อพนักงานสอบสวน" ดังนั้น การร้องทุกข์ของเด็กหญิงตรีผู้เสียหายจึงชอบด้วยกฎหมาย

การสอบสวนชอบด้วยกฎหมายหรือไม่

การสอบสวนไม่ชอบด้วยกฎหมาย เนื่องจากพนักงานสอบสวนไม่ได้ตั้งคําถามเด็กหญิงตรีผู้เสียหายผ่านนักจิตวิทยาหรือนักสังคมสงเคราะห์ ซึ่งขัดต่อประมวลกฎหมายวิธีพิจารณาความอาญา มาตรา 52 ที่ระบุว่า "ในการสอบสวนผู้เสียหายที่เป็นเด็กหรือผู้ที่มีความบกพร่องทางจิตใจ พนักงานสอบสวนต้องจัดให้มีนักจิตวิทยาหรือนักสังคมสงเคราะห์เข้าร่วมสอบสวน" 

นอกจากนี้ พนักงานสอบสวนยังไม่ได้ส่งตัวเด็กหญิงตรีผู้เสียหายไปตรวจหาร่องรอยการกระทําชําเราเป็นพยานหลักฐานทันที ซึ่งขัดต่อประมวลกฎหมายวิธีพิจารณาความอาญา มาตรา 182 ที่ระบุว่า "เมื่อพนักงานสอบสวนได้รับแจ้งเหตุแล้ว ต้องส่งตัวผู้เสียหายไปตรวจหาร่องรอยการกระทําชําเราเป็นพยานหลักฐานทันที"

พนักงานอัยการมีอํานาจฟ้องหรือไม่

พนักงานอัยการมีอํานาจฟ้องนายจัตวาเป็นจําเลยต่อศาลขอให้ลงโทษตามประมวลกฎหมายอาญา มาตรา 277 เนื่องจากพนักงานอัยก